In [ ]:
from importlib.metadata import version
print("torch version:",version("torch"))

torch version: 2.3.0+cu121


In [ ]:
import torch
inputs = torch.tensor([
    [0.43,0.15,0.89],#your
    [0.55,0.87,0.66],#journey
    [0.57,0.85,0.64],#starts
    [0.22,0.58,0.33],#with
    [0.77,0.25,0.10],#one
    [0.05,0.80,0.55] #step
])

Step 1: Compute unnormalized attention scores

attention scores between query x2 and all other input tokens

In [ ]:
query = inputs[1] #2nd input token in query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i,query)

print(attn_scores_2)

dot product is multiplying two vectors element wise and summing resulting products



In [ ]:
res =0
for idx, element in enumerate(inputs[0]):
  res+=inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0],query))

Step 2: normalize unnormalized attention scores so that they sum upto 1


In [ ]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention Weights:",attn_weights_2_tmp)
print("Sum:",attn_weights_2_tmp.sum())

using softmax for normalize is common and recommended

In [ ]:
def softmax_naive(x):
  return torch.exp(x)/torch.exp(x).sum(dim=0)

attn_weights_naive = softmax_naive(attn_scores_2)
print("Attention weights:",attn_weights_naive)
print("Sum:",attn_weights_naive.sum())

In [ ]:
#pytorch softmax function (optimized)
attn_weights_2 = torch.softmax(attn_scores_2,dim=0)
print("attention weights:",attn_weights_2)
print("sum:",attn_weights_2.sum())

compute context vector

---


multiplying input tokens with attention weights and sum resulting vectors





In [ ]:
query = inputs[1] #2nd input token is the query
context_vector_2 = torch.zeros(query.shape[0])
print(context_vector_2)
for i,x_i in enumerate(inputs):
  context_vector_2 += attn_weights_2[i] * x_i
  print(context_vector_2)

computing attention weights for all input tokens

In [ ]:
inputs.shape

torch.Size([6, 3])

In [ ]:
attn_scores = torch.empty(6,6)
for i,x_i in enumerate(inputs):
  for j,x_j in enumerate(inputs):
    attn_scores[i,j]=torch.dot(x_i,x_j)
print(attn_scores)

can achieve the same as above more efficiently using matrix multiplication

In [ ]:
attn_scores = inputs @ inputs.T
print(attn_scores)

In [ ]:
#normalize
attn_weights = torch.softmax(attn_scores,dim=1)
print(attn_weights)

In [ ]:
#values in each row sums to 1
row2_sum = attn_weights[1].sum()
print(row2_sum)
print("all rows sum",attn_weights.sum(dim=-1))

In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

**Implementing self attention with trainable weights**

---




* self attention mechanism used in gpt and other popular llms
* also called scaled dot product attention
* weights matrics updated during training
* three training weight matrices Wq(query), Wk(key), Wv(value)
* input and output dimensions same in gpt but here choose different